In [12]:
import awkward as ak
import numpy as np
from pyutils.pyprocess import Processor
from pyutils.pyselect import Select

In [3]:
branches = ["trksegs", "trksegpars_lh"]

processor = Processor()
data = processor.process_data(
    file_name = "/exp/mu2e/data/users/sgrant/mu2e_cosmic_ana/data/nts.mu2e.CosmicCRYSignalAllOnSpillTriggered.MDC2020au_best_v1_3_v06_03_00.001202_00012335.root",
    branches = branches
)

[pyprocess] ⭐️ Initialised Processor:
	path = 'EventNtuple/ntuple'
	use_remote = False
	verbosity=1
[pyread] ✅ Opened /exp/mu2e/data/users/sgrant/mu2e_cosmic_ana/data/nts.mu2e.CosmicCRYSignalAllOnSpillTriggered.MDC2020au_best_v1_3_v06_03_00.001202_00012335.root
[pyimport] ✅ Imported branches
[pyprocess] ✅ Returning result from process on /exp/mu2e/data/users/sgrant/mu2e_cosmic_ana/data/nts.mu2e.CosmicCRYSignalAllOnSpillTriggered.MDC2020au_best_v1_3_v06_03_00.001202_00012335.root


In [4]:
# Check if C++ assumption of positional correspondence is valid
seg_counts = ak.num(data["trksegs"], axis=-1)
lh_counts = ak.num(data["trksegpars_lh"], axis=-1)

print(f"Tracks where segment counts differ: {ak.sum(seg_counts != lh_counts)}")
print(f"If this is >0, then C++ indexing assumption is definitely wrong")

# Also check if positions of entrance segments vary
entrance_positions = ak.argmax(data["trksegs"]["sid"] == 0, axis=-1)
print(f"Entrance segment positions: {entrance_positions[:20]}")
print(f"If these aren't mostly 0, then first-position assumption is wrong")

Tracks where segment counts differ: 0
If this is >0, then C++ indexing assumption is definitely wrong
Entrance segment positions: [[2, 2, 2, 2], ..., [2, 2, 9, 4, 3, 2, 3, 0, 3, ..., 0, 2, 2, 2, 2, 2, 2, 2, 1]]
If these aren't mostly 0, then first-position assumption is wrong


In [9]:
selector = Select()

at_trk_front = selector.select_surface(data, sid=0)

# Check what at_trk_front actually contains
entrance_positions = ak.argmax(data["trksegs"]["sid"] == 0, axis=-1)
print("Entrance positions:", entrance_positions[:10])

# Check at_trk_front 
at_trk_front_positions = ak.local_index(data["trksegs"], axis=-1)[at_trk_front]
print("at_trk_front positions:", at_trk_front_positions[:10])

# These should match!

[pyselect] ✅ Returning mask for trksegs with sid = 0 and sindex = 0
Entrance positions: [[2, 2, 2, 2], [14, 12, 2, 1, 9, 15, 2, 1], ..., [2, 2, ..., 1], [3, 2, 3, 1]]
at_trk_front positions: [[[2], [2], [2], [2]], [[14], [12], ..., [...], [1]], ..., [[3], [2], [3], [1]]]


In [13]:
# Check for tracks exactly at boundaries
maxr_at_entrance = data["trksegpars_lh"]["maxr"][at_trk_front]
first_maxr = ak.firsts(maxr_at_entrance, axis=-1)
maxr_flat = ak.flatten(first_maxr, axis=None)

exactly_450 = np.sum(maxr_flat == 450.0)
exactly_680 = np.sum(maxr_flat == 680.0)
print(f"Tracks with maxr exactly 450: {exactly_450}")
print(f"Tracks with maxr exactly 680: {exactly_680}")

# Check near-boundary values
near_450 = np.sum(np.abs(maxr_flat - 450.0) < 0.001)
near_680 = np.sum(np.abs(maxr_flat - 680.0) < 0.001)
print(f"Tracks within 0.001 of 450: {near_450}")
print(f"Tracks within 0.001 of 680: {near_680}")

Tracks with maxr exactly 450: 0
Tracks with maxr exactly 680: 0
Tracks within 0.001 of 450: 0
Tracks within 0.001 of 680: 0


In [20]:
# Check array lengths per track
trksegs_len = ak.num(data["trksegs"]["sid"])
lh_len = ak.num(data["trksegpars_lh"]["maxr"])

print("Array length comparison:")
print(f"Same length events: {ak.sum(trksegs_len == lh_len)}")
print(f"Different length events: {ak.sum(trksegs_len != lh_len)}")
print(f"Total events: {len(trksegs_len)}")

# Look at some examples where they differ
mismatched = trksegs_len != lh_len
if ak.any(mismatched):
    print("\nExamples of length mismatches:")
    print(f"trksegs lengths: {trksegs_len[mismatched][:10]}")
    print(f"lh lengths: {lh_len[mismatched][:10]}")

# Check if lh arrays are consistently shorter/longer
print(f"\nEvents where lh is shorter: {ak.sum(lh_len < trksegs_len)}")
print(f"Events where lh is longer: {ak.sum(lh_len > trksegs_len)}")

# Check typical lengths (no flattening needed)
print(f"\nTypical trksegs lengths: {trksegs_len[:20]}")
print(f"Typical lh lengths: {lh_len[:20]}")

# Check if first element alignment makes sense
# Get sid values for first few events
print(f"\nFirst few events - trksegs sid values:")
for i in range(min(5, len(data))):
    if len(data["trksegs"]["sid"][i]) > 0:  # Use len() instead
        print(f"Event {i}: {data['trksegs']['sid'][i]}")

# Also check percentage alignment
matching_percent = 100 * ak.sum(trksegs_len == lh_len) / len(trksegs_len)
print(f"\nPercentage of events with matching lengths: {matching_percent:.1f}%")

Array length comparison:
Same length events: 2653
Different length events: 0
Total events: 2653

Events where lh is shorter: 0
Events where lh is longer: 0

Typical trksegs lengths: [4, 8, 8, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 2, 4, 39]
Typical lh lengths: [4, 8, 8, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 2, 4, 39]

First few events - trksegs sid values:
Event 0: [[96, 101, 0, 1, 2], [2, 1, 0, 101, 96], [96, ..., 2], [2, 1, 0, 101, 96]]
Event 1: [[103, 104, 104, 103, 104, 103, 104, 103, ..., 104, 104, 90, 90, 0, 1, 2], ...]
Event 2: [[101, 90, 0, 1, 2], [101, 90, ..., 1, 2], ..., [1, 0, 90, 90, 101, 90, 90, 0]]
Event 3: [[96, 101, 0, 1, 2], [2, 1, 0, 101, 96], [96, ..., 2], [2, 1, 0, 101, 96]]
Event 4: [[96, 103, 104, 104, 104, 104, 103, 101, 90, 90, 0, 1, 2], ..., [2, 1, ..., 96]]

Percentage of events with matching lengths: 100.0%
